# BERT模型详解

BERT (Bidirectional Encoder Representations from Transformers) 是由Google AI研究团队于2018年提出的预训练语言模型，它彻底改变了自然语言处理领域。

## BERT的核心特点

1. **双向上下文编码**：与GPT等单向模型不同，BERT能够同时考虑文本的左右上下文，捕获更全面的语义信息
2. **基于Transformer编码器**：BERT仅使用Transformer的编码器部分，不包含解码器
3. **预训练-微调范式**：先在大规模无标注文本上预训练，再在下游任务上微调

## 预训练任务

BERT通过两个创新的预训练任务学习语言表示：

1. **掩码语言模型(MLM)**：随机遮盖输入中15%的词元，训练模型预测这些被遮盖的词
2. **下一句预测(NSP)**：训练模型判断两个句子是否为连续的句子对

## 模型规模

BERT有两种主要规格：
- **BERT-Base**：12层Transformer编码器，768维隐藏层，12个注意力头，1.1亿参数
- **BERT-Large**：24层Transformer编码器，1024维隐藏层，16个注意力头，3.4亿参数

## 应用场景

BERT在众多NLP任务中表现出色：
- 文本分类
- 命名实体识别
- 问答系统
- 情感分析
- 文本相似度计算

## BERT的影响与发展

BERT开创了NLP预训练模型的新时代，催生了众多后续模型：
- RoBERTa：优化了BERT的训练方法
- DistilBERT：轻量化版本，保持性能的同时减少参数量
- ALBERT：参数共享技术，减少内存消耗
- ELECTRA：使用判别式预训练方法

BERT的出现标志着NLP领域从特定任务模型向通用预训练模型的重要转变，为后来的大语言模型奠定了基础。


## 代码演示

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# 导入BERT相关代码
from bert import (
    LayerNorm,  # 层归一化
    MultiHeadAttention,  # 多头注意力
    PositionwiseFeedForward,  # 位置前馈网络
    TransformerBlock,  # Transformer块
    BertEmbeddings,  # BERT嵌入层
    BERT,  # BERT模型
    BertForMaskedLM,  # 掩码语言模型
    BertForSequenceClassification,  # 序列分类模型
    BertConfig  # BERT配置
)

# 创建一个小型BERT配置用于演示
config = BertConfig(
    vocab_size=30522,  # 词汇表大小
    hidden_size=768,   # 隐藏层大小
    num_hidden_layers=12,  # Transformer层数
    num_attention_heads=12,  # 注意力头数
    intermediate_size=3072,  # 前馈网络中间层大小
    max_position_embeddings=512,  # 最大位置编码
    type_vocab_size=2,  # 句子类型数量
    dropout_rate=0.1  # 丢弃率
)

# 创建BERT模型
bert_model = BERT(
    vocab_size=config.vocab_size,   # 词汇表大小
    hidden_size=config.hidden_size, # 隐藏层大小
    num_hidden_layers=config.num_hidden_layers, # Transformer层数
    num_attention_heads=config.num_attention_heads, # 注意力头数
    intermediate_size=config.intermediate_size, # 前馈网络中间层大小
    max_position_embeddings=config.max_position_embeddings, # 最大位置编码
    type_vocab_size=config.type_vocab_size, # 句子类型数量
    dropout_rate=config.dropout_rate # 丢弃率
)

# 打印模型结构
print(f"BERT模型参数量: {sum(p.numel() for p in bert_model.parameters()):,}")

# 创建用于掩码语言模型的BERT
mlm_model = BertForMaskedLM(bert_model, config.vocab_size, config.hidden_size)

# 创建用于序列分类的BERT (例如情感分析，2个类别)
cls_model = BertForSequenceClassification(bert_model, num_labels=2, hidden_size=config.hidden_size)

# 演示输入
batch_size = 2
seq_length = 10
input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_length))
token_type_ids = torch.zeros_like(input_ids)
attention_mask = torch.ones_like(input_ids)

# 获取BERT输出
with torch.no_grad():
    bert_output = bert_model(input_ids, token_type_ids, attention_mask)
    mlm_output = mlm_model(input_ids, token_type_ids, attention_mask)
    cls_output = cls_model(input_ids, token_type_ids, attention_mask)

print(f"\nBERT输出形状: {bert_output.shape}")  # [batch_size, seq_length, hidden_size]
print(f"掩码语言模型输出形状: {mlm_output.shape}")  # [batch_size, seq_length, vocab_size]
print(f"序列分类模型输出形状: {cls_output.shape}")  # [batch_size, num_labels]


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.memory_format memory_format = None, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
